# Higher order functions and partial application

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

The topics discussed are:
- our broader goal is to achieve higher and higher levels of abstraction in our code
    - we want to define computational concepts related to our domain _precisely and succintly_
        - this way we can *say more with less*, reducing both effort (mental cost) and chance of mistakes
- we can start by observing two apparently unrelated functions, and still look for similarity between them:

```
sum := n =>
  s := 0
  while n > 0:
    s := s + n
    n := n - 1
  return s

line := n =>
  s := ""
  while n > 0:
    s := s + "*"
    n := n - 1
  return s
```

- of course the two samples are really different, but it is clear that they also share a lot of structure
- let us rewrite them by "abstracting out" the differing bits:

```
sum := n =>
  s := BASE
  while n > 0:
    s := s + STEP(n)
    n := n - 1
  return s

line := n =>
  s := BASE
  while n > 0:
    s := s + STEP(n)
    n := n - 1
  return s
```

- notice that both formulations are now dependent on two extra "things", which we can just define as parameters
    - one of the arguments, `BASE`, is simply a value, so we turn it into an argument
    - the other argument, `STEP`, is a function, but this is no problem: a function can easily be an argument as well!
- **functions are values just like `3` or `5`, and so they can be passed around as parameters, but also returned as results from functions!**
- this leads us to the following definition, generic wrt:
    - the starting value, `BASE`
    - what is added at each step `STEP`, which needs to know which step we are talking about

```
repeat := (n,BASE,STEP) =>
  s := BASE
  while n > 0:
    s := s + STEP(n)
    n := n - 1
  return s
```

- we call `repeat` a higher order function (HOF), since it takes functions as parameters
- we can now instantiate this function twice in order to obtain `sum` and `line`:

```
sum := n => return repeat(n,0,x => return x)
line := n => return repeat(n,"",x => return "*")
```

- `sum` will simply add each value from `n` down to `1`, starting from `0`
- `line` is almost the same: it adds `n` asterisks, starting from the empty string `""`

- both `sum` and `line`, according to the original definition, must take only `n` as a parameter
    - therefore, they both accept `n` as input and pass it through to `repeat` (this is not ideal, and we will improve on this concept later)

- suppose now we added yet another function that we wish to cover under our generic implementation:

```
mul := n =>
  s := 1
  while n > 0:
    s := s * n
    n := n - 1
  return s
```

- unfortunately this function cannot be implemented in terms of `repeat`, because our last implementation only _adds_ the various values together, and `mul` needs to multiply them

- in order to cover `mul` as well, we might generalize `repeat` even more, so that it also covers the operation that we want to perform between steps in order to merge `s` and `STEP(n)`:

```
sum := n =>
  s := BASE
  while n > 0:
    s := MERGE(s, STEP(n))
    n := n - 1
  return s

line := n =>
  s := BASE
  while n > 0:
    s := MERGE(s, STEP(n))
    n := n - 1
  return s

mul := n =>
  s := BASE
  while n > 0:
    s := MERGE(s, STEP(n))
    n := n - 1
  return s
```

- with this more powerful formulation, we could indeed build a very generic `repeat` function that merges `s` and `STEP(n)`, or we could refine it even more
    - we could perform `MERGE` and `STEP` together in a single step, and this new "step which merges" needs to know the current `s` and the step `n`
    - this results in

```
sum := n =>
  s := BASE
  while n > 0:
    s := MERGE_STEP(s, n)
    n := n - 1
  return s

line := n =>
  s := BASE
  while n > 0:
    s := MERGE_STEP(s, n)
    n := n - 1
  return s

mul := n =>
  s := BASE
  while n > 0:
    s := MERGE_STEP(s, n)
    n := n - 1
  return s
```

- this formulation offers a slight advantage: instead of three externally provided parameters, we only need two
- this reduces the amount of context we need to provide

- `repeat` therefore becomes:

```
repeat := (n,BASE,MERGE_STEP) =>
  s := BASE
  while n > 0:
    s := MERGE_STEP(s, n)
    n := n - 1
  return s
```

- and we can simply provide the proper parameters to `sum`, `line`, and `mul` as follows:

```
sum  := n => return repeat(n, 0, (s,n) => return s + n)
mul  := n => return repeat(n, 1, (s,n) => return s * n)
line := n => return repeat(n, "", (s,n) => return s + "*")
```

- we can take even a step further and further improve the definition of `repeat`, since it is clear that `n` is "another sort" of parameter
    - `n` remains a parameter even in the functions `sum`, `mul`, and `line`
- we can turn `repeat` into a function that further returns a function (remember that functions are **just values**)

```
repeat := (BASE,MERGE_STEP) =>
  return n =>
    s := BASE
    while n > 0:
      s = MERGE_STEP(s, n)
      n = n - 1
    return s
```

- now we can specify `BASE` and `MERGE_STEP` as the first parameters, without being forced to provide a fake `n` which we just pass-through:

```
sum  := return repeat(0, (s,n) => return s + n)
mul  := return repeat(1, (s,n) => return s * n)
line := return repeat("", (s,n) => return s + "*")
```

- as one last example, let us consider a very general HOF
    - perhaps the most general
- `then`, to create function pipelines
    - the idea is that given two functions, we can invoke them in sequence in order to create a pipeline of them
    - this is a simple concept, but because of its simplicity and generality it can be used almost anywhere

```
then := (f,g) => return x => g(f(x))
```

- as an example, consider simple functions such as:

```
incr := x => return x + 1
double := x => return x * 2
```

- suppose we wanted to increment twice
    - with `then`, we simply say
```
then(incr,incr)
```

- what about first incrementing, then doubling?

```
then(incr, double)
```

- or even further:

```
then(incr, then(double, incr))
```

- thanks to HOF's, we can create (composable) pipelines of functions, in order to assemble our custom instructions with what effectively becomes a "custom semicolon instruction"
    - our toolbox is not only growing, but our ability to compose concepts together is growing as well!
    - composition is of quintessential importance: without composition, our ability to connect concepts together is limited, and so is our ability to represent _networks of ideas_, which characterize most interesting and articulated problem domains